## Research Questions

1) Does the tone (positive, negative, uncertain) expressed during an earnings call predict short-window abnormal stock returns, defined as the firm’s actual return over the [0, +1]-day event window surrounding the call in excess of the market return (proxied by the S&P 500 ETF, SPY)?
2) Does the tone to return relationship differ across industries, firm sizes, or leadership?
3) Do tone effects weaken or strengthen during high-volatility market days, as measured against the SPY (if there are large increases/decreases in SPY price, are the impacts of tone amplified or dampened)?
4) *Potential question*: After controlling for EPS surprise (the difference between actual returns and the forecasted returns by external analysts, which, when positive or negative, can have a significant impact on a company's stock performance), does tone still explain residual abnormal returns (measured with a [-1,+1] event window)?


## Motivation
Corporate earnings calls serve as the main bridge between enterprises and investors. They shape how markets interpret financial performance beyond the raw numbers, providing context to numeric output. While the quantitative outcomes of an earnings report are easy to measure, the language executives use – be it their tone, confidence, or underlying uncertainty – can carry additional weight, which has the potential to influence investor sentiment when localized to each occurrence.

It is important to study this relationship because, while markets are a quantitative beast, they also rely on narrative, context, and behavioral signals. Prior work has shown that tone effects on “abnormal performance” can be predicted in gradual post-announcement stock price drift. As a contrast, this project isolates the short-window reaction ([0,+1]) to measure the immediate market response to tone, providing a complementary perspective that is clear of other market influences, which conflate analyses. Understanding this aspect of the psychology behind financial decision-making provides an interesting lens into the impacts of behavior on markets, informing future analysis and serving as an input for future models.

Beyond its economic ties, this project provides an interesting computational exploration, combining natural language analysis and statistical models, which is becoming an ever-larger part of financial and economic research. By linking these natural language signals to numerical outcomes, it deepens (my) understanding of how unstructured information can be linked to statistical analysis, and how this information translates to measurable impacts.

Experience with finance and markets, which I have gained over the last 3 years, along with my interest in data science, serve as the foundation for my desire to pursue this project. It is particularly interesting in its combination of NLP and statistical analysis, and I look forward to seeing the results.


## Data Setting
This project draws on three publicly available datasets that together support analysis of how executive tone in earnings calls relates to short-window abnormal stock returns.
1. **[Earnings Call Transcripts (Motley Fool / Kaggle)](https://www.kaggle.com/datasets/tpotterer/motley-fool-scraped-earnings-call-transcripts)** - This dataset includes roughly 18,000 quarterly earnings-call transcripts for U.S.-listed companies. Each record provides the company ticker, call date, exchange, quarter, and full transcript text. The data were scraped from The Motley Fool’s public archives and compiled by Kaggle contributors. The transcripts are the unstructured textual foundation for tone analysis, allowing extraction of sentiment features using finance-specific linguistic dictionaries (see #4).
2. **[NASDAQ Daily Prices (Kaggle / Paul Mooney)](https://www.kaggle.com/datasets/svaningelgem/nasdaq-daily-stock-prices)** - This dataset contains daily open, high, low, close, adjusted-close, and volume (OHLCV) data for U.S. equities from roughly 2015-2024. It enables the computation of firm-level daily returns and the construction of event-window returns surrounding each earnings call date.
3. **[S&P 500 ETF (SPY) Prices (Kaggle)](https://www.kaggle.com/datasets/benjaminbtang/spy-historical-prices)*** - This dataset provides historical daily prices for the SPY ETF, which is used as a market benchmark. Subtracting SPY’s daily return from a firm’s daily return produces a simple measure of abnormal return, controlling for broad market movements.
4. *Supplemental dataset/tool* **|** ***[Loughran-McDonald Financial Sentiment Dictionary](https://sraf.nd.edu/loughranmcdonald-master-dictionary/)*** - Used map word occurrences in transcripts to finance-specific tone categories (positive, negative, uncertainty, etc.). This resource, widely adopted in accounting and finance research, ensures that the tone scores reflect financial meaning rather than generic sentiment.

Each dataset is stored in CSV format and will be merged on ticker and date keys to align firm-level and market-level data for each event window.

#### Potential Challenges
None of the datasets include formal datasheets; however, several contextual details may complicate or encourage deeper analysis:
1. **Coverage and survivorship bias** - The transcript dataset includes only companies covered by The Motley Fool, potentially omitting small-cap or delisted firms. This may over-represent large, stable firms and bias results toward those with stronger disclosure practices.
2. **Timing misalignment** - Earnings calls often take place after market hours, while price data are recorded at the market close. As a result, a “day 0” return may reflect information or expectations formed before the call rather than the call itself, making it important to define the event window ([0,+1]) carefully and account for weekends and holidays.
3. **Linguistic and formatting variation** - Transcripts differ in speaker labeling, punctuation, and inclusion of boilerplate disclaimers or operator remarks. These inconsistencies may distort tone-scoring unless the text is systematically cleaned.


## Method
Step 1: Load and prepare data
- Load the three datasets (earnings call transcripts, stock prices, and SPY benchmark) using pandas
- Standardize date formats and align all data by ticker and date.
- Functions (data-manipulation): load_data(), standardize_dates()
- Tests: Use small 3–5 row samples to confirm correct data types and successful merges.
- Output: Three clean DataFrames with properly formatted and aligned dates.
- Connection: Establishes base for the Multiple Datasets challenge goal by merging separate data sources.


Step 2: Clean transcripts and compute tone features
- Use Python’s built-in re library to remove punctuation, lowercase text, and normalize spacing.
- Count occurrences of positive, negative, and uncertainty words using the Loughran–McDonald financial dictionary.
- Calculate each tone category as a percentage of total words in the transcript.
- Functions (data-manipulation): clean_text(), compute_tone_scores()
- Tests: Verify results on short sample texts (“profits increased,” “uncertain outlook”) with known word counts.
- Output: Dataset with tone metrics (pos_pct, neg_pct, uncert_pct) for each earnings call.
- Connection: Creates the independent variables used in hypothesis testing (RQ1 and RQ2).


Step 3: Compute event-window and abnormal returns
- Compute daily returns for each stock and for SPY using adjusted close prices.
-Define Day 0 as the first trading day on or after the call date and Day +1 as the following trading day.
- Calculate abnormal returns as firm return minus SPY return, then sum over [0,+1] to get cumulative abnormal return (CAR).
- Functions (data-manipulation): compute_returns(), compute_abnormal_returns()
- Tests: Hand-check results on a small, synthetic dataset to confirm correct math and event-window handling.
- Output: Event-level dataset linking each call to its short-window abnormal return.
- Connection: Provides the dependent variable for statistical testing and supports Multiple Datasets.

Step 4: Merge tone and return data
- Merge tone metrics with event returns and add basic controls such as sector and firm-size proxies.
- Functions (data-manipulation): merge_features(), add_controls()
- Tests: Ensure one row per event after merging and confirm correct ticker/date alignment.
- Output: Combined dataset ready for modeling.
- Connection: Prepares data for hypothesis testing (RQ1 and RQ2).

Step 5: Hypothesis testing and modeling
- Run regression models using statsmodels to test whether tone predicts short-term abnormal returns:
     car_0p1 ~ pos_pct + neg_pct + uncert_pct + sector + size_proxy
- Evaluate coefficients, p-values, and confidence intervals to test significance.
- Adjust for multiple comparisons (e.g., Benjamini–Hochberg correction) if running across multiple tone types or sectors.
- Functions (data-manipulation): fit_model(), summarize_results()
- Tests: Use synthetic data with known relationships to confirm correct coefficient direction and model behavior.
- Interpretation:
- - RQ1: Positive coefficients on pos_pct or negative on neg_pct indicate tone predicts abnormal returns.
- - RQ2: Interaction terms or coefficient differences by sector suggest heterogeneity.
- Connection: Directly achieves the Statistical Hypothesis Testing challenge goal.

Step 6: Visualization
- Create plots to display tone distributions, tone vs. return relationships, and regression coefficients.
- Functions (plotting): plot_tone_vs_returns(), plot_coefficients()
- Tests: No formal testing; figures checked visually for accuracy and clarity.
- Output: Visual confirmation of tone–return relationships.
- Connection: Helps interpret quantitative results for RQ1 and RQ2.


*Step 7: Robustness and reporting (optional)*
- *Re-run models using alternative event windows ([−1,+1] or [0,+5]) to confirm consistency.*
- *Winsorize extreme returns to check for sensitivity to outliers.*
- *Save outputs, figures, and summary tables for reporting.*
- *Connection: Provides robustness checks for RQ1 and RQ3, ensuring conclusions are not window-dependent.*

#### Plan

The project will be completed in JupyterHub and divided into five main tasks, each designed to be clear, independent, and reproducible.


1) Setup and data preparation (2 hours): I will create an organized folder structure in JupyterHub with subfolders for raw data, processed data, figures, and reports. After confirming the environment setup, I will load the earnings call transcripts, stock prices, and SPY benchmark data using pandas. During this step, I will standardize date formats, check for missing or duplicated keys, and ensure that tickers and dates align across datasets to prepare for merging.


2) Text cleaning and tone computation (3 hours): Using Python’s re library, I will remove punctuation, normalize spacing, and lowercase the transcript text. I will then apply the Loughran-McDonald financial dictionary or the spaCy API to calculate the percentage of positive, negative, and uncertainty words for each transcript. The resulting tone features will be saved as a separate dataset and tested on a small subset of text examples to confirm accuracy.


3) Return calculations and event-window construction (3 hours): I will compute daily returns for both individual tickers and the SPY benchmark. For each earnings call, I will define the event window as [0,+1], where Day 0 represents the first trading day on or after the call. Abnormal returns will be calculated as the firm’s return minus SPY’s return, and cumulative abnormal returns (CAR) will be saved for each event. Manual checks on a small synthetic dataset will verify the accuracy of these calculations.


4) Merging, modeling, and hypothesis testing (5 hours): I will merge the tone dataset with abnormal returns and add control variables such as industry sector and firm size proxies (e.g., log of average volume). Using statsmodels, I will run regression models to test whether tone predicts short-window abnormal returns while controlling for other factors. I will interpret coefficients, p-values, and confidence intervals directly in the context of the research questions.


5) Visualization and reporting (~3 hours): The final step will involve creating plots to display the distribution of tone features, the relationship between tone and abnormal returns, and regression coefficients with confidence intervals. If time allows, I will perform quick robustness checks such as alternate event windows or light outlier filtering. All intermediate results, figures, and tables will be saved for reproducibility.


*This plan builts in buffer time and may be an overestimation*



## EDA Results

lalalla

In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import pickle
import doctest
from pathlib import Path
from IPython.display import display
from lmd_loader import load_masterdictionary # from custom python file
from pandas import DataFrame # for type annotations

### Initial Preparation and Analysis

#### I/O
Function(s) for loading

In [2]:
# Earning Reports - cannot limit pickle read
def read_er_pkl(path: str, head_rows=None) -> DataFrame:
    '''Load the earnings report pickle file into a df.'''
    with open(path, 'rb') as file:
        df = pickle.load(file)
    return df.head(head_rows) if head_rows else df

In [20]:
# # NASDAQ OHLCV
# def read_nasdaq_folder(folder_path, pattern='*csv', limit_files=None, nrows=None):
#     '''
#     '''
#     files = list(Path(folder_path).glob(pattern))
#     if limit_files is not None:
#         files = files[:limit_files]
#     df_list = [pd.read_csv(file, nrows=nrows) for file in files]
#     return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()

In [125]:
# files = list(Path('data/nasdaq_prices').glob('*.csv'))
# len(files)

1377

#### Data Transforms
Functions used in inital loading/cleaning

In [3]:
# For NASDAQ OHLCV + SPY Date Conversions

def date_transform(df: DataFrame, src_col: str, out_col: str) -> DataFrame:
    '''
    Converts a column from string to datetime and stores it under a new name.
    '''
    df[out_col] = pd.to_datetime(df[src_col])
    df[out_col] = df[out_col].dt.normalize()   # precautionary check

    return df

In [4]:
# Earning Reports Date Conversion

# ** does the .loc work properly? need to make sure that it is only updating dates that have a true bool value

def er_date_transform(df: DataFrame, src_col: str ='date', out_col: str ='date_std') -> DataFrame:
    '''
    Given a dataset, returns a dataset with a cleaned and standardized date column. Receives 
    an input for the source date column, and the name for the outputted standardized date column.
    '''
    date_clean = df[src_col].str.strip()
    date_clean = date_clean.str.replace(".","")
    date_clean = date_clean.str.replace("ET","")
    
    df[out_col] = pd.to_datetime(date_clean, format='mixed')

    # boolean mask for past 5:00 pm identification -> works because indexes line up etc
    after_close = df[out_col].dt.hour >= 17
    row_updates = df.loc[after_close, out_col]            # stores all (masked)
    updated_dates = row_updates + pd.Timedelta(days=1)    # uses pandas timedelta function to add 1 day to the date
    df.loc[after_close, 'date_std'] = updated_dates       # uses .loc for conditional selection base on True | False values in after_close
    df[out_col] = df[out_col].dt.normalize()              # strip times for use with 

    return df

#### File Calls
Main file loading, with constrained loading phases to address memory constraints (4GB)

##### SPY

In [5]:
spy_df = pd.read_csv('data/SPY.csv')
spy_df = date_transform(spy_df, 'Date', 'date_std')

##### LMD (Sentiment Dict) implementation
Implemented using the lmd_loader.py script (pulled from the [official script](https://drive.google.com/file/d/18jbZ3o17PRI_s4xG9UslKnGMpnC1ZoLM/view))

In [6]:
# Sentiment Dictionary
lmd_path = 'data/Loughran-McDonald_MasterDictionary_1993-2024.csv'

# load everything
# vars identified in return stmnt at bottom of LMD loader, incl params for logging
master_dict, md_header, sentiment_categories, sentiment_dicts, stopwords, total_docs = \
    load_masterdictionary(lmd_path, print_flag=True, f_log=None, get_other=True)

 ...Loading Master Dictionary 85,000
Master Dictionary loaded from file:
  data/Loughran-McDonald_MasterDictionary_1993-2024.csv

  master_dictionary has 86,553 words.



In [7]:
sentiment_categories

['negative',
 'positive',
 'uncertainty',
 'litigious',
 'strong_modal',
 'weak_modal',
 'constraining',
 'complexity']

##### Earnings Reports / Sentiment Analysis
Must perform sentiment analysis in loading phase due to memory constraints

In [8]:
def lmd_features(text: str, sentiment_dicts: dict[str, dict[str, int]], 
                 cat_order: list[str] = sentiment_categories) -> dict[str, float]:
    '''
    Computes proportion of words in input text that belong to each LMD sentiment
    category. For every category (pre-loaded by sentiment dict), it counts the 
    number of matching words and produces a proportion of the total for that word.
    Designed to avoid creating large intermediate lists. 
    '''
    if not isinstance(text, str): # checks for string to prevent errors
        if text is None:
            text = ""
        else:
            text = str(text)
    
    upper_text = text.upper() # normalization

    # declares the counter and assigns base 0s
    counts = {}
    for category in cat_order:
        counts[category] = 0

    text_convert = re.compile(r"[A-Za-z]+")
    total = 0

    # ** using advanced RE module for memory efficiency **
    iterator = text_convert.finditer(upper_text) # identifies matches in the string for iteration

    # ? efficiency notes due to massive memory usage
    
    # match being a container that has text, start position, end position
    for match in iterator:                        # "for words in text" = n words = O(n)
        total += 1
        tok = match.group(0) 
        
        for category in cat_order:                # for category in list of categories -> 9 cat.
            cat_dict = sentiment_dicts[category]
            if tok in cat_dict:                   # 'if' search through a dict = O(1)
                counts[category] += 1
    
    if total < 1:
        denom = 1           # avoids division by 0 errors
    else:
        denom = total
    
    # computes the percentages for each category
        # does not work for syllables (would need avg syllables)
        # -> must ensure the syllable column is dropped
    result = {}
    for category in cat_order: 
        result[category] = counts[category] / denom
    
    return result

In [11]:
# feature tests
# Test 1: Positive text
result = lmd_features("able abundance", sentiment_dicts, sentiment_categories)
assert result['positive'] > 0, "X Should detect positive words"
assert result['negative'] == 0, "X Should have no negative words"
print("Y Test 1: Positive text")

# Test 2: Negative text
result = lmd_features("loss decline", sentiment_dicts, sentiment_categories)
assert result['negative'] > 0, "X Should detect negative words"
assert result['positive'] == 0, "X Should have no positive words"
print("Y Test 2: Negative text")

# Test 3: Empty string
result = lmd_features("", sentiment_dicts, sentiment_categories)
assert result['positive'] == 0, "X Empty should be 0"
assert result['negative'] == 0, "X Empty should be 0"
print("Y Test 3: Empty string")

# Test 4: None input
result = lmd_features(None, sentiment_dicts, sentiment_categories)
assert result['positive'] == 0, "X None should be handled"
print("Y Test 4: None input")

Y Test 1: Positive text
Y Test 2: Negative text
Y Test 3: Empty string
Y Test 4: None input


In [9]:
def apply_features(df, text_col="transcript", drop_text=True):
    """
    Add sentiment columns to dataframe. **finsish
    ** add type annotations
    """
    # initial gaurdrail
    if text_col not in df.columns:
        raise KeyError(f"Column '{text_col}' not in DataFrame")
    
    # applies sentiment function to each transcript (s = the 'current' transcript)
    feat_dicts = df[text_col].apply(
        lambda s: lmd_features(s, sentiment_dicts, sentiment_categories))
    
    # .apply converts dicts into columns/series -> then appended
    feats = feat_dicts.apply(pd.Series)
    
    # applies transcript drops (per parameter) -- can make this one line in final
    if drop_text:
        base = df.drop(columns=[text_col])
    else:
        base = df
        
    # concat original and features
    out = pd.concat([base, feats], axis=1)
    
    # testing
    for cat in sentiment_categories:
        assert cat in out.columns, "Columns missing"
        print("Columns present")
        assert (out[cat].between(0, 1)).all(), "Improper values"
        print("Values eligible")
    
    return out

In [10]:
# Earnings Reports
er_df = read_er_pkl("data/motley-fool-data.pkl")
er_df = er_date_transform(er_df, 'date', 'date_std')
er_df = er_df.dropna(subset=['date_std'])
# below: removes the time stamps -> not included in function for separate use case later
# --> use times for adjusted window calcs in final
# ** er_df['date_std'] = er_df['date_std'].dt.date

# applies tokenizer below
er_df = apply_features(er_df, 'transcript', True)

Columns present
Values eligible
Columns present
Values eligible
Columns present
Values eligible
Columns present
Values eligible
Columns present
Values eligible
Columns present
Values eligible
Columns present
Values eligible
Columns present
Values eligible


##### NASDAQ OHLCV
Simplified loading process that reduces memory usage by focusing on relevant tickers

In [12]:
def load_single_ohlcv(file: str) -> DataFrame:
    """
    Load a single OHLCV file with ticker column, barring
    ticker column errors, and returns the dataframe.
    """
    df = pd.read_csv(file)
    if 'ticker' not in df.columns:
        df['ticker'] = file.stem.upper()
    
    return df

In [13]:
def load_relevant_ohlcv(ohlcv_folder: str, er_df: DataFrame) -> DataFrame:
    '''
    Returns a df with cols [ticker, date_std, open, high, low, close],
    after loading OHLCV data only for tickers that appear in the inputted
    earnings report (reducing storage significantly).
    '''

    # get unique tickers using set()
    rel_tickers = set(er_df['ticker'].str.upper()) # standardized to string and upper for comparison
    print(f"Found {len(rel_tickers)} unique tickers in earnings reports")

    # uses path module/object with .glob to get all file paths, and place in list
    files = list(Path(ohlcv_folder).glob('*.csv'))

    # filters file paths to only retain 'relevant' tickers
    rel_files = []
    for file in files:
        # pulls ticker names using the .stem()
        ticker = file.stem.upper()
        if ticker in rel_tickers:
            rel_files.append(file)
    
    print(f"{len(rel_files)} overlapping tickers in OHLCV")

    # full file loading using prev-built loading function
    df_list = []
    for file in rel_files:
        df_list.append(load_single_ohlcv(file))

    # takes the list of df, concats into one, and normalizes dates
    combined_df = pd.concat(df_list, ignore_index=True)
    combined_df = date_transform(combined_df, 'date', 'date_std')
    
    # filter columns for export
    cols_to_keep = ['ticker', 'date_std', 'open', 'high', 'low', 'close']
    combined_df = combined_df[cols_to_keep]

    return combined_df

In [14]:
ohlcv_df = load_relevant_ohlcv('data/nasdaq_prices', er_df)
ohlcv_df.head()

Found 2869 unique tickers in earnings reports
425 overlapping tickers in OHLCV


,ticker,date_std,open,high,low,close
0,AXGN,1986-12-17,0.0,4.10,4.00,4.00
1,AXGN,1986-12-18,0.0,4.10,4.00,4.00
2,AXGN,1986-12-19,0.0,4.10,4.00,4.00
3,AXGN,1986-12-22,0.0,4.10,4.00,4.00
4,AXGN,1986-12-23,0.0,3.89,3.67,3.67


#### *Tests/Checks - Initial load and cleaning*

In [15]:
assert 'date_std' in er_df.columns
assert 'ticker' in er_df.columns

assert 'date_std' in ohlcv_df.columns
assert 'ticker' in ohlcv_df.columns
assert 'date_std' in spy_df.columns

#### Stock Return Calcs
*Similar Function

In [16]:
def daily_returns_calc(df: DataFrame, price_col: str, date_col: str, group_col: str, return_col: str) -> DataFrame:
    """
    Computes daily returns given an input df, price col, date col, and grouping column,
    using a simple percent change formula. Returns a df that retains the calculated
    returns, and that is sorted ensuring congruent calculations. 
    """
    # asserts for input verification (are they necessary?)
    assert price_col in df.columns, f"Column '{price_col}' not found"
    assert date_col in df.columns, f"Column '{date_col}' not found"
    assert len(df) > 0, "DataFrame cannot be empty"
    
    new_df = df.copy()      # could avoid the copy for memory improvement, but use if possible

    # data should be sorted, but pct change calc will be WRONG if it isnt, so we ensure
    # group_col must be specified as None in the call if performing for a single stock -> 
        # risk too high if accidentally forgetten with multi ticker data
    if group_col is None:
        new_df = new_df.sort_values(date_col)
        new_df[return_col] = new_df[price_col].pct_change()
    else:
        sort_cols = [group_col, date_col]
        new_df = new_df.sort_values(sort_cols)
        new_df[return_col] = new_df.groupby(group_col)[price_col].pct_change()

    assert new_df[return_col].notna().sum() > 0, "All returns are NaN | 0"

    return new_df

In [17]:
ohlcv_returns = daily_returns_calc(ohlcv_df, 'close', 'date_std', 'ticker', 'return_fract')
spy_returns = daily_returns_calc(spy_df, 'Close', 'date_std', group_col = None, return_col='spy_return_fract')

#### Window Return Calculations
Window return calc needed pre-merge, due to data loss on date filter
- Ensure the window factors in weekend etc
- Then call the window calc function for different day variations (i.e. 1d, 2d, 5d)
- Bring in something else to actually get the real drift outside of surprise??

In [17]:
# def event_window_car(merged_df: DataFrame, window_days: int = 2) -> DataFrame:
#     """
#     Computes cumulative abnormal return (CAR) over [0, +1] event window.
#     Abnormal return = stock return - SPY return
    
#     **NOTE** This is simplified - assumes next calendar day = next trading day;
#     full version will handle weekends/holidays properly.
#     """
#     out = merged_df.copy()
    
#     # For each earnings call, gets returns on day 0 and day +1
#     # Simplified v1: just gets  abnormal return on the call date
#     out['abnormal_return'] = out['return'] - out['spy_return']
    
#     # **EDA using single-day abnormal return as proxy for CAR**
#     out['car_0p1'] = out['abnormal_return']  # Placeholder
    
#     return out

In [ ]:
def window_return(df: DataFrame, window_period):
    """
    """
    return_col = f"return_{window_period}d"

#### Data Merges

In [18]:
# uses inner join to merge pricing and report data (w ticker + date filter)
merged = er_df.merge(ohlcv_returns, on=['ticker', 'date_std'], how='inner')
merged.head()

,date,exchange,q,ticker,date_std,negative,positive,uncertainty,litigious,strong_modal,weak_modal,constraining,complexity,open,high,low,close,return_fract
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,2020-08-28,0.004299,0.022356,0.006879,0.001376,0.006191,0.002752,0.000688,0.000172,45.7100,48.8600,45.5100,47.3900,0.049845
1,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,2019-11-06,0.007390,0.011442,0.010965,0.001073,0.002384,0.004648,0.000596,0.000596,86.9544,90.3949,84.1391,88.6084,0.087974
2,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,2019-08-07,0.009237,0.019979,0.009667,0.000000,0.003437,0.005800,0.000859,0.002793,12.5193,14.6062,12.0945,14.5777,0.083417
3,"Nov 06, 2019, 4:30 p.m. ET",NASDAQ: DXCM,2019-Q3,DXCM,2019-11-06,0.006592,0.023799,0.007263,0.001341,0.005251,0.003464,0.001676,0.001229,38.4975,39.3138,38.0662,38.2800,-0.002865
4,"Feb 10, 2021, 9:00 a.m. ET",NASDAQ: EEFT,2020-Q4,EEFT,2021-02-10,0.006983,0.015793,0.006983,0.000752,0.003653,0.004620,0.002578,0.003223,136.9200,145.3400,136.6900,141.1000,0.010600


### Q: How large is the dataset?
The datasets combine three complementary sources: corporate earnings-call transcripts, daily stock prices, and a market benchmark. The earnings-report data includes 18,755 rows and 6 columns, where each row represents a single company’s earnings call on a specific date. Columns contain identifiers (ticker, date_std), the full transcript_text,  sentiment token scores, and supporting metadata such as company name or file origin. This dataset serves as the textual foundation for tone analysis, capturing the language used by executives during calls.

The market price data (OHLCV) comprises 133,557 rows and 7 columns, with each row corresponding to one trading-day observation for a given firm (delineated by ticker). Columns record standard financial attributes: open, high, low, close, and volume. These values will later be used to calculate daily and event-window returns. For the benchmark dataset, it is drawn from the S&P 500 ETF (SPY), includes 7,703 rows and 8 columns, with each row representing one trading day for the broader market. It mirrors the OHLCV structure (with the addition of adj_close) and provides the baseline for measuring abnormal returns.

Note: In this EDA, the OHLCV table was filtered immediately at load time (file-level limitation) for memory control purposes -> i.e., we did not first load the full datset and then filter by transcript overlap, like we will in later iterations.

In [27]:
# data checks
required_cols = {
    "er_df": {"ticker", "date_std"},
    "ohlcv_df": {"ticker", "date_std"},
    "spy_df": {"date_std"},
}

dfs = [("er_df", er_df), ("ohlcv_df", ohlcv_df), ("spy_df", spy_df)]

for name, df in dfs:
    missing = required_cols[name] - set(df.columns)
    assert not missing, f"{name} missing columns: {missing}"
    assert pd.api.types.is_datetime64_any_dtype(df["date_std"]), f"{name} date_std must be datetime"

In [72]:
# data size summary
print(f"Merged stock data shape: {merged.shape[0]} rows × {merged.shape[1]} cols")
print(f"SPY shape: {spy_df.shape[0]} rows × {spy_df.shape[1]} cols")

print("Merged data preview:")
display(merged.head(3))
print("SPY preview:")
display(spy_df.head(3))

Merged stock data shape: 2837 rows × 17 cols
SPY shape: 7703 rows × 8 cols
Merged data preview:


,date,exchange,q,ticker,date_std,negative,positive,uncertainty,litigious,strong_modal,weak_modal,constraining,complexity,open,high,low,close
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,2020-08-28,0.004299,0.022356,0.006879,0.001376,0.006191,0.002752,0.000688,0.000172,45.7100,48.8600,45.5100,47.3900
1,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,2019-11-06,0.007390,0.011442,0.010965,0.001073,0.002384,0.004648,0.000596,0.000596,86.9544,90.3949,84.1391,88.6084
2,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,2019-08-07,0.009237,0.019979,0.009667,0.000000,0.003437,0.005800,0.000859,0.002793,12.5193,14.6062,12.0945,14.5777


SPY preview:


,Date,Open,High,Low,Close,Adj Close,Volume,date_std
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,25.029377,1003200,1993-01-29
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,25.207405,480500,1993-02-01
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,25.260784,201300,1993-02-02


### Does the dataset have any missing data?

Looking at the datasets, the missingness analysis shows that only the Earnings Reports (ER) dataset contained missing data. Specifically, the column date_std had 380 missing values, accounting for approximately 2.03% of all rows. These missing entries likely stem from parsing issues or inconsistent metadata in the raw date column that prevented some dates from being successfully standardized. Because the date_std column is a critical key for aligning transcripts with market data and defining event windows, these rows cannot be reliably used in downstream analysis.

For this exploratory phase, we will remove the affected rows, as their proportion is small enough that exclusion will not be materially affecting results or representativeness of the data. The OHLCV and SPY datasets showed no missing values, indicating that the market data and benchmark series are structurally complete and ready for use in return computations. In future iterations, we may implement a more robust date-parsing procedure to recover these records rather than discarding them.

In [30]:
def missing_data(df: pd.DataFrame, name: str) -> pd.DataFrame:
    '''
    Return a DataFrame of columns with missing values and their %.
    Always returns columns: ['missing_count','missing_pct'] -> they 
    may be empty.
    '''
    total = df.shape[0]
    missing = df.isna().sum()
    missing = missing[missing > 0]

    if missing.empty:
        print(f"{name}: no missing values")
        return missing.to_frame("missing_count").assign(
            missing_pct=pd.Series(dtype=float)) # so that DF is returned even if there are no missing vals

    result = (
        missing.to_frame("missing_count") 
        # .assign  adds a column in a chain - lambda=take the chained df and calc percent in new col
        .assign(missing_pct=lambda x: (x["missing_count"] / total * 100).round(2))
        .sort_values("missing_pct", ascending=False)) 

    print(f"{name} missing data:")
    display(result)
    print()
    return result

In [31]:
er_miss    = missing_data(er_df,    "Earnings Reports (ER)")
ohlcv_miss   = missing_data(ohlcv_df,   "OHLCV")
spy_miss   = missing_data(spy_df,   "SPY")

Earnings Reports (ER) missing data:


,missing_count,missing_pct
date_std,380,2.03



OHLCV: no missing values
SPY: no missing values


### Variables of Interest
Across the three datasets, the variables of interest collectively span the corporate language seen in earnings reports to market behavior. In the Earnings Reports (ER) data, each observation corresponds to one firm’s earnings call (ticker, date_std), with the transcript text (transcript) processed into quantitative tone features. token_count measures call length, while pos_pct, neg_pct, and uncert_pct represent the normalized share of positive, negative, and uncertainty-related words, respectively. These tone variables provide the linguistic inputs for assessing whether executive sentiment has an influence on short-term price reactions.

The OHLCV dataset records daily market activity for each firm, including open, high, low, and close, as well as computed daily returns (ret) from adjusted closing prices. This data quantifies firm-level market response, serving as the behavioral side of the tone–return relationship.

The SPY dataset represents the benchmark market index, structured identically to OHLCV but aggregated at the market level. Its daily return (ret) acts as the baseline for calculating abnormal returns. Together, these variables form a coherent system: linguistic tone captures firm-level communication sentiment, OHLCV captures firm-specific market behavior, and SPY provides the market context necessary for comparison.

In [113]:
er_summary = er_df[["token_count","pos_pct","neg_pct","uncert_pct"]].describe().T
display(er_summary)


,count,mean,std,min,25%,50%,75%,max
token_count,18755.0,7863.904452,2511.699771,562.0,6121.000000,7961.000000,9502.500000,32286.000000
pos_pct,18755.0,0.005711,0.002916,0.0,0.003540,0.005352,0.007455,0.025175
neg_pct,18755.0,0.001069,0.000984,0.0,0.000458,0.000816,0.001374,0.011607
uncert_pct,18755.0,0.004685,0.001582,0.0,0.003583,0.004496,0.005588,0.021007


In [116]:
ohlcv_summary = ohlcv_df[["open","high","low","close"]].describe().T
display(ohlcv_summary)

,count,mean,std,min,25%,50%,75%,max
open,133557.0,4.119939e+07,1.538108e+09,0.0000,4.78,10.5900,25.50,9.281250e+10
high,133557.0,4.301171e+07,1.608392e+09,0.0274,5.49,11.2418,27.40,9.585000e+10
low,133557.0,3.898297e+07,1.450471e+09,0.0210,5.13,10.6874,25.55,8.100000e+10
close,133557.0,4.142139e+07,1.546763e+09,0.0222,5.30,10.9300,26.50,9.180000e+10


In [119]:
spy_summary = spy_df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].describe().T
display(spy_summary)

,count,mean,std,min,25%,50%,75%,max
Open,7703.0,1.675022e+02,1.033269e+02,43.343750,1.047700e+02,1.316400e+02,2.088650e+02,4.792200e+02
High,7703.0,1.685138e+02,1.038956e+02,43.531250,1.056125e+02,1.324400e+02,2.097850e+02,4.799800e+02
Low,7703.0,1.663896e+02,1.027221e+02,42.812500,1.038500e+02,1.306800e+02,2.078300e+02,4.760600e+02
Close,7703.0,1.675101e+02,1.033526e+02,43.406250,1.048800e+02,1.315600e+02,2.088500e+02,4.777100e+02
Adj Close,7703.0,1.398787e+02,1.094186e+02,24.726746,7.082888e+01,9.370667e+01,1.812353e+02,4.665634e+02
Volume,7703.0,8.451404e+07,9.280526e+07,5200.000000,9.742550e+06,6.263260e+07,1.168936e+08,8.710263e+08


### Challenge Goals:
The challenge goal for this EDA was to extend beyond transcript length and incorporate actual tone measures into the analysis. This was achieved by implementing an in-notebook sentiment analysis using a subset of the Loughran–McDonald dictionary to create normalized tone variables (pos_pct, neg_pct, uncert_pct). While these are prototype features for EDA, the structure directly supports future hypothesis testing and will scale easily when the full dictionary is applied for final analysis. No further challenge extensions (e.g., multi-dataset joins) in order to maintain focus on the core.

### Plan Evaluation
The initial work plan proved fairly accurate in scope: dataset loading and cleaning took roughly the estimated time, while tone extraction and EDA each required slightly longer due to testing and validation. The decision to keep the datasets separate simplified memory use and avoided alignment errors. The current tasks completed - shape validation, missingness, tone feature generation, and visual summaries - align closely with the planned timeline. Remaining tasks include finalizing hypothesis testing and abnormal-return calculation, estimated to require one additional work session. Overall, the plan has remained realistic and on track.

### Testing Overview (proof)
Testing approach: All of the transformations feeding the EDA were validated with assertions and small doctests. The tone_counts() function was manually verified with sample strings to confirm correct tokenization and sentiment counting. Key tests include verifying that token_count >= 0, tone percentages lie within [0,1], and date_std is properly formatted as datetime. Plotting functions were not directly tested per rubric, but their inputs were validated through descriptive statistics and shape checks. Together, these confirm that the EDA results can be trusted and replicated on the full dataset.

In [118]:
spy_df.columns.tolist()


['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'date_std']

### Extras:
##### Table Join

In [115]:
er['ticker'].isin(ohlcv['ticker']).value_counts()

ticker
False    15832
True      2923
Name: count, dtype: int64

In [18]:
def ticker_compare (df1, col1, df2, col2):
    '''
    Given two dataframes and two column names, returns the values shared
    between the two dataframe columns, for each dataframe (returned as a
    tuple of two separate dataframes).

    >>> df1 = pd.DataFrame({'ticker': ['AAPL', 'MSFT', 'GOOG'], 'price': [100, 200, 300]})
    >>> df2 = pd.DataFrame({'ticker': ['AAPL', 'TSLA'], 'text': ['apple er', 'tesla er']})
    >>> df1_common, df2_common = ticker_compare(df1, 'ticker', df2, 'ticker')
    >>> sorted(df1_common['ticker'].unique())
    ['AAPL']
    >>> sorted(df1_common['price'].unique())
    [100]
    >>> sorted(df2_common['ticker'].unique())
    ['AAPL']

    >>> df3 = pd.DataFrame({'ticker': ['AMZN'], 'close': [150]})
    >>> df4 = pd.DataFrame({'ticker': ['NFLX'], 'text': ['netflix']})
    >>> df3_common, df4_common = ticker_compare(df3, 'ticker', df4, 'ticker')
    >>> len(df3_common)
    0
    >>> len(df4_common)
    0
    >>> sorted(df2_common['text'].unique())
    ['apple er']
    '''
    ticker_1 = set(df1[col1])
    ticker_2 = set(df2[col2])
    shared = ticker_1 & ticker_2

    df1_common = df1[df1[col1].isin(shared)]
    df2_common = df2[df2[col2].isin(shared)]

    return df1_common, df2_common

doctest.run_docstring_examples(ticker_compare, globals())

In [19]:
ohlcv_common, er_common = ticker_compare(ohlcv, 'ticker', er, 'ticker')

In [20]:
ohlcv_common.describe()

,open,high,low,close,date_std
count,300.000000,300.000000,300.000000,300.000000,300
mean,9.208552,9.468413,8.923588,9.201829,2008-02-29 22:38:24
min,2.500000,2.670000,2.420000,2.580000,1993-06-11 00:00:00
25%,4.250000,4.407500,4.080000,4.387500,1993-09-27 18:00:00
50%,10.622800,10.774850,10.411800,10.601200,2012-06-02 12:00:00
75%,11.417500,11.668275,11.160000,11.350000,2018-05-02 06:00:00
max,20.800000,22.700000,19.600000,21.280000,2018-08-17 00:00:00
std,4.411799,4.588522,4.216676,4.373212,NaN


In [1]:
   # %history -n -l 100
